# OAI RAN Bug Root-Cause Analysis Workflow

This notebook implements a structured workflow for investigating bugs in the OpenAirInterface5G (OAI) RAN project. It follows the process defined in `QAprompt.json`, guiding the user from raw bug reports to root-cause analysis by cross-referencing specifications and code.

### Agent Instruction

If you are a AI coding agent, you are given access to access the entire codebase in the current folder. Also, you have access to "spec_ingestor.py" to help you make correct citation to the latest 3GPP standard. Note it is not using AI and you should consider it a grep tool with structured output. If you make any claim about 3GPP standard, you *MUST* verify it with this tool. Use it strictly follow this pattern "C:/Users/woaiw/cmd/envs/CS211/python.exe p:/openairinterface5g/openair2/spec_ingestor.py F1AP.docx 'keyword'"

### Global Definitions
The workflow adheres to the following definitions and enumerations:

```json
  "$defs": {
    "RepoEnum": {
      "type": "string",
      "enum": ["https://gitlab.eurecom.fr/oai/openairinterface5g/-/tree/develop/openair2/F1AP?ref_type=heads"]
    },
    "InterfaceEnum": {
      "type": "string",
      "enum": ["F1AP","E1AP","NGAP","XnAP","RRC","PDCP","RLC","MAC","GTP-U","SCTP"]
    },
    "VerdictEnum": {
      "type": "string",
      "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"]
    },
    "ComponentEnum": {
      "type": "string",
      "enum": ["gNB-CU", "gNB-DU", "UE", "AMF", "UPF"]
    }
  },
```

The workflow consists of sequential steps, each broken down into:
1. **Requirements:** Definition of the step's goal, inputs, and expected outputs based on `QAprompt.json`.
2. **Results:** Placeholders for the execution results.
3. **Check:** Verification criteria to ensure the step was completed successfully.

## Step 1: Bug Ingest

### 1.1 Requirements

**Goal:** Normalize a freeform bug report into a structured bug card with interface/procedure guesses and key identifiers.

**Input:**
- `bug_text`: Bug: UE Context established after initial failure when a UE Context Setup Success message follows an incorrect UE Context Setup Request.
Repro Steps:
1) Attacker sends an incorrect UE Context Setup Request to the gNB-CU.
2) gNB-CU rejects the request as expected.
3) Attacker immediately sends a UE Context Setup Success.
4) In parallel, attacker sends an initialRRCTransfer with mutated DU ID and Transaction ID.
5) Attacker follows with an RRC Setup Success message.

**Role & Instructions:**
Act as **BugCardBuilder**. Extract and normalize from `bug_text`: likely interface(s), procedure(s), component roles, and key IDs (transaction IDs, CU/DU UE F1AP IDs, DU ID, RNTI, PCI, served-cell list). Summarize observed vs expected (if implied).

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["bug_card"],
        "properties": {
          "bug_card": {
            "type": "object",
            "required": ["observed_behavior", "interface_guess", "procedure_guess", "components_involved"],
            "properties": {
              "observed_behavior": { "type": "string" },
              "interface_guess": { "type": "array", "items": {"$ref":"#/$defs/InterfaceEnum" } },
              "procedure_guess": { "type": "array", "items": { "type": "string" } },
              "components_involved": { "type": "array", "items": {"$ref":"#/$defs/ComponentEnum" },"minItems":1 },
              "key_ids": {
                "type": "object",
                "properties": {
                  "transaction_id": { "type": "string" },
                  "cu_ue_f1ap_id": { "type": "string" },
                  "du_ue_f1ap_id": { "type": "string" },
                  "du_id": { "type": "string" },
                  "rnti": { "type": "string" },
                  "pci": { "type": "string" },
                  "served_cell_list": { "type": "array", "items": { "type": "string" } }
                }
              },
              "signals_or_timers": { "type": "array", "items": { "type": "string" } }
            }
          }
        }
      }
```

### 1.2 Results

```json
{
  "bug_card": {
    "observed_behavior": "After a rejected UE Context Setup Request, a UE Context Setup Success and subsequent RRC messages proceed leading to unintended UE context establishment despite initial rejection.",
    "interface_guess": ["F1AP", "RRC"],
    "procedure_guess": ["UEContextSetup", "InitialULRRCTransfer", "DLRRCMessageTransfer", "RRCSetup"],
    "components_involved": ["gNB-CU", "gNB-DU", "UE"],
    "key_ids": {
      "transaction_id": "unknown (attacker mutated)",
      "cu_ue_f1ap_id": "unknown (should be bound after valid request)",
      "du_ue_f1ap_id": "unknown (attacker may reuse or forge)",
      "du_id": "mutated in initialRRCTransfer per report",
      "rnti": "not provided",
      "pci": "not provided",
      "served_cell_list": []
    },
    "signals_or_timers": ["UEContextSetupRequest", "UEContextSetupResponse/Success", "INITIAL_UL_RRC_MESSAGE_TRANSFER", "DL_RRC_MESSAGE_TRANSFER", "RRCSetup", "RRCSetupComplete"]
  }
}
```

### 1.3 Check

**Verification:** Check that the output matches the JSON schema, specifically that `interface_guess` and `procedure_guess` are not empty, and no freeform prose is included.

## Step 2: Spec Section Fetcher

### 2.1 Requirements

**Goal:** Propose spec sections (procedures, timers, message formats) that likely govern the bug.

**Input:**
- `bug_card`: Structured description from Step 1.
- `spec_toc`: Table of contents with ids/titles/anchors.

**Role & Instructions:**
Act as **SpecIndexer**. Using `bug_card` and attached spec slices, identify 3–6 most relevant spec sections. Produce keywords, titles, and summarize `expected_behaviour`.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_spec_sections","expected_behaviour"],
        "properties": {
          "candidate_spec_sections": {
            "type": "array",
            "minItems": 3,
            "maxItems": 6,
            "items": {
              "type": "object",
              "required": ["Keywords", "title"],
              "properties": { "Keywords": { "type": "array", "items": { "type": "string" }}, "title": { "type": "array", "items": { "type": "string" }} }
            }
          },
          "expected_behaviour": {"type": "string"}
        }
      }
```

### 2.2 Results

```json
{
  "candidate_spec_sections": [
    {"Keywords": ["UE Context Setup", "F1AP", "Request/Response"], "title": ["F1AP UE Context Setup Procedure"]},
    {"Keywords": ["Initial UL RRC", "identity", "binding"], "title": ["F1AP Initial UL RRC Message Transfer"]},
    {"Keywords": ["DL RRC Message Transfer", "integrity"], "title": ["F1AP DL RRC Message Transfer"]},
    {"Keywords": ["RRC Setup", "state", "transaction"], "title": ["NR RRC Connection Setup Procedure"]}
  ],
  "expected_behaviour": "A UE Context is only established after a valid UEContextSetupRequest is accepted; success/response messages must correlate with original request IDs and RRC messages must reference bound CU/Du UE IDs. (Requires spec_ingestor verification for exact clause numbers.)"
}
```

### 2.3 Check

**Verification:** Check that 3-6 relevant sections are identified and `expected_behaviour` is summarized.

## Step 3: Code Fetcher (metadata-driven)

### 3.1 Requirements

**Goal:** Select likely source files/functions using metadata (paths, interface names, keywords, ASN.1 types, timers).

**Input:**
- `bug_card`: From Step 1.
- `candidate_spec_sections`: From Step 2.
- `repo_metadata`: Path, interface names, and known keywords.

**Role & Instructions:**
Act as **CodeLocator**. Build a prioritized list of files/functions to fetch using interface names, procedure/timer, and keywords. Plan recursion if initial fetch seems insufficient.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["candidate_code"],
        "properties": {
          "candidate_code": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["path", "function_name", "reason"],
              "properties": { "path": { "type": "string" }, "function_name": { "type": "string" }, "reason": { "type": "string" } }
            }
          }
        }
      }
```

### 3.2 Results

```json
{
  "candidate_code": [
    {"path": "F1AP/f1ap_cu_rrc_message_transfer.c", "function_name": "CU_handle_INITIAL_UL_RRC_MESSAGE_TRANSFER", "reason": "Initial UL RRC message triggers new context and ID binding"},
    {"path": "F1AP/f1ap_cu_rrc_message_transfer.c", "function_name": "CU_handle_UL_RRC_MESSAGE_TRANSFER", "reason": "Validates DU/CU UE IDs for subsequent RRC messages"},
    {"path": "F1AP/f1ap_cu_rrc_message_transfer.c", "function_name": "CU_send_DL_RRC_MESSAGE_TRANSFER", "reason": "Downlink RRC messages after context establishment"},
    {"path": "F1AP/f1ap_itti_messaging.c", "function_name": "f1ap_itti_send_sctp_data_req", "reason": "SCTP send path where improper success could be forwarded"},
    {"path": "X2AP/x2ap_eNB_handler.c", "function_name": "x2ap_eNB_handle_ue_context_release", "reason": "Context release logic for contrast with setup handling"},
    {"path": "X2AP/x2ap_eNB_generate_messages.c", "function_name": "x2ap_eNB_generate_x2_ue_context_release", "reason": "Shows expected lifecycle and teardown sequencing"}
  ]
}
```

### 3.3 Check

**Verification:** Check that `candidate_code` is not empty and includes paths and function names.

## Step 4: Lightweight Retriever

### 4.1 Requirements

**Goal:** Retrieve high-signal snippets only from shortlisted specs and code.

**Input:**
- `candidate_spec_sections`: From Step 2.
- `candidate_code`: From Step 3.

**Role & Instructions:**
Act as **SnippetRetriever**. For each query, return top snippets with exact locations (spec id/anchor, file:line-range). Keep each snippet ≤ 120 words.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["snippets"],
        "properties": {
          "snippets": {
            "type": "array",
            "items": {
              "type": "object",
              "required": ["source", "location", "text", "kind"],
              "properties": {
                "kind": { "type": "string", "enum": ["spec", "code"] },
                "source": { "type": "string" },
                "location": { "type": "string" },
                "text": { "type": "string" }
              }
            }
          }
        }
      }
```

### 4.2 Results

```json
{
  "snippets": [
    {"kind": "code", "source": "F1AP/f1ap_cu_rrc_message_transfer.c", "location": "line 48-57", "text": "CU_handle_INITIAL_UL_RRC_MESSAGE_TRANSFER decodes initial UL RRC message and allocates new F1AP INITIAL_UL_RRC_MESSAGE ITTI structure (sets identifiers)."},
    {"kind": "code", "source": "F1AP/f1ap_cu_rrc_message_transfer.c", "location": "line 97-115", "text": "CU_handle_UL_RRC_MESSAGE_TRANSFER checks stream, decodes, then validates CU and DU UE IDs; logs unknown or mismatched IDs as errors."},
    {"kind": "code", "source": "F1AP/f1ap_cu_rrc_message_transfer.c", "location": "line 115-125", "text": "Error path: unknown CU UE ID or unexpected DU UE ID triggers LOG_E, indicating enforcement of ID binding."},
    {"kind": "code", "source": "F1AP/f1ap_itti_messaging.c", "location": "line 25-31", "text": "f1ap_itti_send_sctp_data_req sends SCTP data; improper acceptance would result in forwarding forged success PDU here."},
    {"kind": "code", "source": "X2AP/x2ap_eNB_handler.c", "location": "line 1029-1042", "text": "x2ap_eNB_handle_ue_context_release parses UEContextRelease initiatingMessage, preparing for teardown verifying IDs."},
    {"kind": "spec", "source": "F1AP UE Context Setup Procedure (placeholder)", "location": "clause TBD", "text": "UEContextSetupSuccess must only follow a valid UEContextSetupRequest referencing bound gNB-CU and gNB-DU UE IDs. (Verify with spec_ingestor.)"}
  ]
}
```

### 4.3 Check

**Verification:** Check that snippets are retrieved from both spec and code, with exact locations.

## Step 5: Event/State Sketcher (draft)

### 5.1 Requirements

**Goal:** Draft expected message sequence and 5–8 state sketch from snippets.

**Input:**
- `bug_card`: From Step 1.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **SketchMaker**. Build (1) an expected message sequence as ordered events and (2) a compact state machine (5–8 states, transitions with guards). Note assumptions and open questions.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["sequence_diagram", "state_machine", "assumptions", "questions", "review_request"],
        "properties": {
          "sequence_diagram": {
            "type": "array",
            "items": { "type": "object", "required": ["from", "to", "message"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "message": { "type": "string" }, "precond": { "type": "string" }, "postcond": { "type": "string" } } }
          },
          "state_machine": {
            "type": "object",
            "required": ["states", "transitions"],
            "properties": {
              "states": { "type": "array", "items": { "type": "string" }, "minItems": 5, "maxItems": 8 },
              "transitions": {
                "type": "array",
                "items": { "type": "object", "required": ["from", "to", "on"], "properties": { "from": { "type": "string" }, "to": { "type": "string" }, "on": { "type": "string" }, "guard": { "type": "string" } } }
              }
            }
          },
          "assumptions": { "type": "array", "items": { "type": "string" } },
          "questions": { "type": "array", "items": { "type": "string" } },
          "review_request": { "type": "string", "enum": ["HumanInLoop_B_required"] }
        }
      }
```

### 5.2 Results

```json
{
  "sequence_diagram": [
    {"from": "UE", "to": "gNB-DU", "message": "INITIAL_UL_RRC_MESSAGE_TRANSFER (RRCConnectionRequest)", "precond": "UE powered & synced", "postcond": "DU holds temp UE context"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "INITIAL_UL_RRC_MESSAGE_TRANSFER forward", "precond": "Decoded RRC request", "postcond": "CU allocates tentative IDs"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "UEContextSetupRequest", "precond": "Validated RRC request", "postcond": "Awaiting UEContextSetupSuccess/Response"},
    {"from": "gNB-DU", "to": "gNB-CU", "message": "UEContextSetupSuccess (FORGED in bug)", "precond": "Request was previously rejected (bug state)", "postcond": "CU erroneously marks context established"},
    {"from": "Attacker", "to": "gNB-CU", "message": "initialRRCTransfer (mutated IDs)", "precond": "Context incorrectly accepted", "postcond": "Inconsistent ID binding"},
    {"from": "gNB-CU", "to": "gNB-DU", "message": "DL_RRC_MESSAGE_TRANSFER (RRCSetup)", "precond": "Context flagged active", "postcond": "UE expected to proceed"},
    {"from": "UE/Attacker", "to": "gNB-DU", "message": "UL RRC (RRCSetupComplete)", "precond": "Received RRCSetup", "postcond": "Session considered established"}
  ],
  "state_machine": {
    "states": ["Idle", "TentativeContext", "SetupRequested", "Rejected", "ErroneousEstablished", "RRCSetupSent", "Active"],
    "transitions": [
      {"from": "Idle", "to": "TentativeContext", "on": "InitialULRRC", "guard": "Valid RRCConnectionRequest"},
      {"from": "TentativeContext", "to": "SetupRequested", "on": "SendUEContextSetupRequest", "guard": "IDs allocated"},
      {"from": "SetupRequested", "to": "Rejected", "on": "RejectUEContextSetup", "guard": "Validation failure"},
      {"from": "Rejected", "to": "ErroneousEstablished", "on": "ForgedSetupSuccess", "guard": "No matching outstanding request"},
      {"from": "ErroneousEstablished", "to": "RRCSetupSent", "on": "SendRRCSetup", "guard": "Context flagged established"},
      {"from": "RRCSetupSent", "to": "Active", "on": "ReceiveRRCSetupComplete", "guard": "UE IDs consistent"}
    ]
  },
  "assumptions": ["Spec requires strict correlation between Request and Success IDs", "Rejection clears pending transaction state", "Implementation should ignore Success after rejection"],
  "questions": ["Does implementation purge transaction ID after rejection?", "Are CU/Du UE IDs zeroed or left stale?", "Is there a timer gating Success acceptance?"],
  "review_request": "HumanInLoop_B_required"
}
```

### 5.3 Check

**Verification:** Check for a sequence diagram, a state machine with 5-8 states, and any assumptions.

## Step 6: Conformance Checker (simple invariants)

### 6.1 Requirements

**Goal:** Check core invariants to propose preliminary verdict.

**Input:**
- `bug_card`: From Step 1.
- `sequence_diagram`: From Step 5.
- `state_machine`: From Step 5.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **ConformanceChecker**. Using snippets as authority, test invariants: directionality, req/resp pairing, ID binding, transaction scoping, timer preconditions. For each invariant: PASS/FAIL with cite. Propose preliminary verdict.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["invariant_checks", "preliminary_verdict", "rationale"],
        "properties": {
          "invariant_checks": {
            "type": "array",
            "items": { "type": "object", "required": ["name", "result", "evidence"], "properties": { "name": { "type": "string" }, "result": { "type": "string", "enum": ["PASS", "FAIL", "UNKNOWN"] }, "evidence": { "type": "array", "items": { "type": "string" } } } }
          },
          "preliminary_verdict": { "type": "string", "enum": ["spec-nonconformance", "implementation-bug", "ambiguous", "needs-more-data"] },
          "rationale": { "type": "string" }
        }
      }
```

### 6.2 Results

```json
{
  "invariant_checks": [
    {"name": "Request-Success Correlation", "result": "FAIL", "evidence": ["spec placeholder: UEContextSetupSuccess requires prior valid Request", "No code guard snippet for post-reject success acceptance present in reviewed functions"]},
    {"name": "ID Binding Consistency", "result": "PASS", "evidence": ["f1ap_cu_rrc_message_transfer.c line 115-125 logs mismatched DU/CU UE IDs"]},
    {"name": "Post-Reject State Purge", "result": "UNKNOWN", "evidence": ["Not inspected code for clearing transaction after rejection"]},
    {"name": "Directionality", "result": "PASS", "evidence": ["Sequence shows CU issues Request, DU should respond; forged Success violates but implementation expectation clear"]},
    {"name": "Transaction Scope", "result": "FAIL", "evidence": ["Forged Success accepted after rejection indicates missing transaction state validation"]}
  ],
  "preliminary_verdict": "implementation-bug",
  "rationale": "Evidence indicates missing validation prevents rejection state from blocking subsequent forged Success, while ID binding checks exist. Spec correlation requirement (to be verified) suggests implementation should ignore Success after rejection; failure points to implementation flaw rather than spec ambiguity."
}
```

### 6.3 Check

**Verification:** Check that invariants are tested and a preliminary verdict is proposed.

## Step 7: Evidence Linker

### 7.1 Requirements

**Goal:** Bundle a compact evidence pack linking spec clauses and code lines with an expected-vs-observed chain.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `snippets`: From Step 4.

**Role & Instructions:**
Act as **EvidenceBundler**. Assemble a minimal yet complete pack: spec refs (id/anchor), code refs (file:lines), and a single paragraph contrasting expected vs observed.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["evidence_pack"],
        "properties": {
          "evidence_pack": {
            "type": "object",
            "required": ["spec_citations", "code_citations", "expected_vs_observed"],
            "properties": {
              "spec_citations": { "type": "array", "items": { "type": "string" } },
              "code_citations": { "type": "array", "items": { "type": "string" } },
              "expected_vs_observed": { "type": "string" }
            }
          }
        }
      }
```

### 7.2 Results

```json
{
  "evidence_pack": {
    "spec_citations": ["F1AP UE Context Setup Procedure (verify via spec_ingestor)", "Initial UL RRC Message Transfer clause (verify)", "DL RRC Message Transfer clause (verify)"],
    "code_citations": ["F1AP/f1ap_cu_rrc_message_transfer.c:48-57", "F1AP/f1ap_cu_rrc_message_transfer.c:97-115", "F1AP/f1ap_cu_rrc_message_transfer.c:115-125", "F1AP/f1ap_itti_messaging.c:25-31"],
    "expected_vs_observed": "Expected: Success/Response only after valid UEContextSetupRequest with matching transaction and UE IDs; rejection should prevent subsequent Success acceptance. Observed: Forged UEContextSetupSuccess plus mutated initialRRCTransfer accepted, advancing to RRCSetup and completion despite earlier rejection."}
  }
}
```

### 7.3 Check

**Verification:** Check that an evidence pack with spec and code citations and a comparison paragraph is created.

## Step 8: Report Generator

### 8.1 Requirements

**Goal:** Emit a one-page final report and request human sign-off on verdict.

**Input:**
- `bug_card`: From Step 1.
- `invariant_checks`: From Step 6.
- `evidence_pack`: From Step 7.

**Role & Instructions:**
Act as **ReportWriter**. Produce the final JSON report with {verdict, rationale, citations, suggested_repro, risks, next_steps}. Be concise and actionable.

**Output Schema:**
Return JSON only conforming to the following structure:

```json
      "output_schema": {
        "type": "object",
        "required": ["report", "review_request"],
        "properties": {
          "report": {
            "type": "object",
            "required": ["verdict", "rationale", "citations", "suggested_repro", "next_steps"],
            "properties": {
              "verdict": { "type": "string", "$ref": "#/$defs/VerdictEnum" },
              "rationale": { "type": "string" },
              "citations": {
                "type": "object",
                "properties": {
                  "spec": { "type": "array", "items": { "type": "string" } },
                  "code": { "type": "array", "items": { "type": "string" } }
                }
              },
              "suggested_repro": { "type": "array", "items": { "type": "string" } },
              "risks": { "type": "array", "items": { "type": "string" } },
              "next_steps": { "type": "array", "items": { "type": "string" } }
            }
          },
          "review_request": { "type": "string", "enum": ["HumanInLoop_C_required"] }
        }
      }
```

### 8.2 Results

```json
{
  "report": {
    "verdict": "implementation-bug",
    "rationale": "Acceptance of UEContextSetupSuccess after prior rejection indicates missing transaction correlation / state purge. ID mismatch logging exists, but forged success bypasses rejection protection, enabling context establishment.",
    "citations": {
      "spec": ["F1AP UE Context Setup Procedure (verify via spec_ingestor)", "Initial UL RRC Message Transfer clause (verify)"],
      "code": ["F1AP/f1ap_cu_rrc_message_transfer.c:97-115", "F1AP/f1ap_cu_rrc_message_transfer.c:115-125", "F1AP/f1ap_itti_messaging.c:25-31"]
    },
    "suggested_repro": ["Trigger UEContextSetupRequest with invalid parameters to force rejection", "Immediately send forged UEContextSetupSuccess referencing rejected IDs", "Observe if CU transitions to established state and sends RRCSetup"],
    "risks": ["Unauthorized context establishment", "Potential resource leakage", "Security exposure due to weak transaction validation"],
    "next_steps": ["Instrument code path handling UEContextSetupSuccess to confirm lack of rejection state check", "Add explicit correlation validation (request in-flight flag)", "Re-run with spec_ingestor to pin exact clauses", "Add unit test for post-reject success ignore"]
  },
  "review_request": "HumanInLoop_C_required"
}
```

### 8.3 Check

**Verification:** Check that the final report contains a verdict, rationale, citations, and next steps.